### Test Data Analysis

In [1]:
# start_time.describe()

In [2]:
# mpl.rcParams['figure.figsize'] = (6, 6)
# start_time.hist()
# # Start time is from 4 30 am to 7 30 am

In [3]:
# end_time.describe()

In [4]:
# end_time.hist()
# # end time is 8am - 5pm

In [5]:
# length.describe()

In [6]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [7]:
# !nvidia-smi

## Split data and Normalize

In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [9]:
cleaned_df = pd.read_csv('cleaned_df.csv')
imputed_df = pd.read_csv('imputed_df.csv')

In [10]:
from datetime import date
def calc_time(time):
    return int(time.split(':')[0])*60 + int(time.split(':')[1])
def calc_day(day):
    x = date(int(day.split('/')[2]), int(day.split('/')[0]),int(day.split('/')[1]))
    if(day.split('/')[2]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[2]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[2]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[2]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[2]=='2021'):
        return x.timetuple().tm_yday+365*3+1
def test_calc_day(day):
    x = date(int(day.split('/')[0]), int(day.split('/')[1]),int(day.split('/')[2]))
    if(day.split('/')[0]=='2017'):
        return x.timetuple().tm_yday-365+1
    if(day.split('/')[0]=='2018'):
        return x.timetuple().tm_yday
    if(day.split('/')[0]=='2019'):
        return x.timetuple().tm_yday+365
    if(day.split('/')[0]=='2020'):
        return x.timetuple().tm_yday+365*2
    if(day.split('/')[0]=='2021'):
        return x.timetuple().tm_yday+365*3+1

In [11]:
del_days_list = ['3/9/2019', '11/17/2020']

del_days_list = [calc_day(i) for i in del_days_list]

del_days_list


[433, 1052]

In [12]:
df_list = [None]*300
for i in range(1,301):
    t_df = pd.read_csv('data_to_nvidia/Data_Test/{}/weather_data.csv'.format(i))
    t_df['Time'] = [calc_time(j) for j in t_df['MST']]
    t_df['day'] = [test_calc_day(j) for j in t_df['DATE (YYYY/MM/DD)']]
    df_list[i-1] = t_df

In [13]:
start_time = []
end_time = []
length = []
test_days_list = []
for t_df in df_list:
    start_time.append(t_df['Time'][0])
    length.append(len(t_df))
    end_time.append(t_df['Time'][len(t_df)-1])
    test_days_list.append(t_df['day'][0])
start_time = pd.Series(start_time)
end_time = pd.Series(end_time)
length = pd.Series(length)

test_days_list_unique = list(pd.Series(test_days_list).unique())

In [14]:
def fetch(tup):
    day,start_time,end_time = tup
    df = imputed_df[imputed_df['day']==day]
    return df.loc[(df['Time'] >= start_time) & (df['Time'] <= end_time)].reset_index(drop=True)

In [15]:
# Split into Train and Validation data

X = imputed_df['day'].unique()
from sklearn.utils import shuffle
X = shuffle(X, random_state = 9)
X = list(X)
for i in test_days_list_unique:
    X.remove(i)
for i in del_days_list:
    X.remove(i)
train_days = X[0:int(len(X)*0.85)]
val_days = X[int(len(X)*0.85):]
test_days = test_days_list_unique

def get_dataframe(dataf,days_list):
  indexes_list = []
  for day in days_list:
    indexes_list = np.append(indexes_list, dataf[dataf['day'] == day].index)
  return dataf.iloc[np.array(indexes_list).astype('int64')]

In [16]:
train_df = get_dataframe(imputed_df, train_days).drop(['MST','Date'], axis = 1)
val_df = get_dataframe(imputed_df, val_days).drop(['MST','Date'], axis = 1)
test_df = get_dataframe(imputed_df, test_days).drop(['MST','Date'], axis = 1)

a = train_df['day']
b = val_df['day']
c = test_df['day']

In [17]:
# Normalize the data
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df- train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std
train_df['day'] = a
val_df['day'] = b
test_df['day'] = c

In [18]:
full_test_dfs = [fetch(i).drop(['MST','Date'], axis = 1) for i in list(zip(test_days_list,start_time,end_time))]

for i in range(len(full_test_dfs)):
       full_test_dfs[i] = (full_test_dfs[i]- train_mean) / train_std

In [19]:
def create_submission(x,path):
  x.index = list(range(1,301))
  x.to_csv(path,header=['10_min_horizon','20_min_horizon','30_min_horizon','40_min_horizon','50_min_horizon', '60_min_horizon','70_min_horizon','80_min_horizon','90_min_horizon','100_min_horizon','110_min_horizon','120_min_horizon'], index=True, index_label = 'scenario_set')

In [20]:
train_std

GHI                  300.476921
DNI                  402.490428
Zenith Angle          19.014482
Azimuth Angle         65.313841
Dry Temp              10.829638
Wet Temp               7.037837
Dew Temp               7.930834
RH                    24.293388
TCC                   35.166801
Pressure               5.456585
Precipitation          2.356805
Snow Depth             2.938195
Moisture               0.252206
Albedo                 0.172038
Time                 217.275696
day                  414.287369
Declination Angle      0.000090
distance factor        0.693841
Wind_x                 1.814109
Wind_y                 3.066635
Clear Sky GHI        276.027177
dtype: float64

### Using Deep Learning Models

In [21]:
# # extract test data
# import os
# test_data = []*300
# for i in range(1,301):
#   t_df = pd.read_csv(os.path.join('/content/drive/MyDrive/Shell_ai/dataset/test', str(i), "weather_data.csv")) 
#   t_df['Total Cloud Cover [%]'].replace(-7999, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-1, np.NaN, inplace  = True)
#   t_df['Total Cloud Cover [%]'].replace(-6999, np.NaN, inplace  = True) 
#   t_df['Total Cloud Cover [%]'].fillna(method="ffill", inplace=True)
#   t_df['Total Cloud Cover [%]'].fillna(method='backfill', inplace=True)
#   t_df.drop('Time [Mins]', axis = 1, inplace = True)
#   wv = t_df.pop('Peak Wind Speed @ 6ft [m/s]')
#   # Convert to radians.
#   wd_rad = t_df.pop('Avg Wind Direction @ 6ft [deg from N]')*np.pi / 180
#   # Calculate the wind x and y components.
#   t_df['Wind_x'] = wv*np.cos(wd_rad)
#   t_df['Wind_y'] = wv*np.sin(wd_rad)
#   t_df = (t_df- train_mean.drop('day')) / train_std.drop('day')
#   test_data.append(t_df)

In [22]:
import tensorflow as tf
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tf.random.set_seed(42)
np.random.seed(42)

In [23]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,label_shift,
               train_df = train_df, val_df = val_df, test_df = test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift
    self.label_shift = label_shift
    self.total_window_size = input_width + shift + label_shift*(label_width - 1)

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.input_width + self.shift - 1
    self.labels_slice = slice(self.label_start, None, label_shift)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])
    
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=128)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

import random
def chain(iterables):
  while True:
    random.shuffle(iterables)
    for it in iterables:
      lst = list(it)
      random.shuffle(lst)
      for element in lst:
          yield element

In [69]:
def DATALOADER(data, y, bs):
    i = 0
    length = len(y)
    while True:
        batch_data = []
#         batch_imgs = []
        batch_labels = []
        while len(batch_labels) < bs:
              batch_data.append(data[i])
#               batch_imgs.append(imgs[i])
              batch_labels.append(y[i])
              i += 1
              if i >= length:                    # Reset if reached end of paths
                  i = 0
#                   np.random.shuffle(data)
#                   np.random.shuffle(imgs)
#                   np.random.shuffle(y)
                  shuffler = np.random.permutation(len(y))
                  data = data[shuffler]
                  y = y[shuffler]
        # You can call one method inside another method using self.method_name(arugments)
        # Note that you don't have to pass "self" while calling the method, just the remaining arguments  

        # yield the batch to the calling function
        yield (np.array(batch_data), np.array(batch_labels))

### Wandb

In [25]:
import wandb
from tensorflow import keras
from wandb.keras import WandbCallback
wandb.login()

wandb: Currently logged in as: akhila (use `wandb login --relogin` to force relogin)


True

In [26]:
# model.built = True
l1 = ['GHI',
 'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'Dew Temp',
 'RH',
 'TCC',
 'Pressure',
 'Precipitation',
 'Snow Depth',
 'Moisture',
 'Albedo',
 'day',
 'Declination Angle',
 'distance factor',
 'Wind_x',
 'Wind_y',
 'Clear Sky GHI']

l2 = ['GHI',
 'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'Dew Temp',
 'RH',
 'TCC',
 'Pressure',
 'Precipitation',
 'Moisture',
 'Wind_x',
 'Wind_y',
 'distance factor',
 'Clear Sky GHI']

l3 = ['GHI',
       'DNI',
 'Azimuth Angle',
 'Dry Temp',
 'RH',
 'Pressure',
 'TCC',
 'Precipitation',
 'distance factor',
 'Clear Sky GHI']


l4 = ['GHI',
 'Azimuth Angle',
 'TCC',
 'distance factor',
 'Clear Sky GHI']

l = [l1,l2,l3,l4]


### MLP

In [27]:
def Model(units_1,units_2):

    mlp_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = units_1, activation='leaky_relu'),
    tf.keras.layers.Dense(units = units_2, activation='leaky_relu'),
    tf.keras.layers.Dense(units=12),
  ])
    return mlp_model

In [28]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 2,
      'eta': 2
    },

  'parameters': {
      'learning_rate':{
          'values': [1e-2]
      },
      'sr_nd_ws':{
          'values': [[10,12]]
      },
      'units_1':{
          'values': [256]
      },
      'units_2':{
          'values': [32]
      },
      'cols_list':{
          'values': [2,3]
      },
      'optimizers':{
          'values': [0]
      },
      'loss':{
          'values': [0]
      }
    },
                     
 }

In [29]:
def sweep_train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'learning_rate': 0.001
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep
    cfg = wandb.config
    # Specify the other hyperparameters to the configuration, if any
#     cfg.epochs = 30
#     cfg.architecture_name = "Huber_MLP"

    grouped_df1 = train_df.groupby(['day'], sort = False, as_index = False)
    iter_list_train = [ WindowGenerator(input_width=cfg.sr_nd_ws[1], label_width = 12, shift=10//cfg.sr_nd_ws[0], label_shift = 10//cfg.sr_nd_ws[0],train_df = grouped_df1.get_group(day)[::cfg.sr_nd_ws[0]][l[cfg.cols_list]],label_columns=['GHI']).train for day in train_df['day'].unique() ]

    grouped_df1 = val_df.groupby(['day'], sort = False, as_index = False)
    iter_list_val = [ WindowGenerator(input_width=cfg.sr_nd_ws[1], label_width = 12, shift=10//cfg.sr_nd_ws[0], label_shift = 10//cfg.sr_nd_ws[0],train_df = grouped_df1.get_group(day)[::cfg.sr_nd_ws[0]][l[cfg.cols_list]],label_columns=['GHI']).train for day in val_df['day'].unique() ]

    grouped_df1 = test_df.groupby(['day'], sort = False, as_index = False)
    iter_list_test = [ WindowGenerator(input_width=cfg.sr_nd_ws[1], label_width = 12, shift=10//cfg.sr_nd_ws[0], label_shift = 10//cfg.sr_nd_ws[0],train_df = grouped_df1.get_group(day)[::cfg.sr_nd_ws[0]][l[cfg.cols_list]],label_columns=['GHI']).train for day in test_df['day'].unique() ]

    train_batches = 0
    for x in iter_list_train:
      train_batches = train_batches + len(x)
    print(train_batches)
    val_batches = 0
    for x in iter_list_val:
      val_batches = val_batches + len(x)
    print(val_batches)

    test_batches = 0
    for x in iter_list_test:
      test_batches = test_batches + len(x)
    print(test_batches)

    train_gen = chain(iter_list_train)
    val_gen = chain(iter_list_val)
    test_gen = chain(iter_list_test)

    # initialize model
    model = Model(cfg.units_1,cfg.units_2)
    
#     sgd = tf.keras.optimizers.SGD(lr=cfg.learning_rate, momentum = 0.9)
#     adam = tf.keras.optimizers.Adam(lr=cfg.learning_rate)
#     opt = [sgd,adam]
#     losses = ['mae','mse']
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=cfg.learning_rate, momentum = 0.9),loss=tf.keras.losses.Huber(),metrics=[tf.metrics.MeanAbsoluteError()])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience = 3,  mode='min'),
        ReduceLROnPlateau(factor=0.25, patience = 2, verbose=1),
        ModelCheckpoint('MLP_l:{}_sr:{}_ws:{}_u1:{}_u2:{}_cols:{}.h5'.format(cfg.learning_rate, cfg.sr_nd_ws[0],cfg.sr_nd_ws[1], cfg.units_1, cfg.units_2,cfg.cols_list), verbose=1, save_best_only=True, save_weights_only=True) ]

  
    # train and validate
    history = model.fit(train_gen, steps_per_epoch = train_batches, epochs=30, validation_data = val_gen, validation_steps = val_batches,callbacks = [WandbCallback(),callbacks])

In [30]:
sweep_id = wandb.sweep(sweep_config, project="shell_hack-level-2")

Create sweep with ID: yhgu2efk
Sweep URL: https://wandb.ai/akhila/shell_hack-level-2/sweeps/yhgu2efk


In [31]:
wandb.agent(sweep_id, function=sweep_train, count = 48)

wandb: Agent Starting Run: 0tqoceqp with config:
wandb: 	cols_list: 2
wandb: 	learning_rate: 0.01
wandb: 	loss: 0
wandb: 	optimizers: 0
wandb: 	sr_nd_ws: [10, 12]
wandb: 	units_1: 256
wandb: 	units_2: 32


wandb: Ctrl + C detected. Stopping sweep.


In [32]:
sr = 10
for ws in [12]:
    grouped_df1 = train_df.groupby(['day'], sort = False, as_index = False)
    iter_list_train = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in train_df['day'].unique() ]

    grouped_df1 = val_df.groupby(['day'], sort = False, as_index = False)
    iter_list_val = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in val_df['day'].unique() ]

    grouped_df1 = test_df.groupby(['day'], sort = False, as_index = False)
    iter_list_test = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in test_df['day'].unique() ]
    # test_generator = itertools.chain.from_iterable( iter_list )

    train_batches = 0
    for x in iter_list_train:
        train_batches = train_batches + len(x)
    print(train_batches)
    val_batches = 0
    for x in iter_list_val:
        val_batches = val_batches + len(x)
    print(val_batches)

    test_batches = 0
    for x in iter_list_test:
        test_batches = test_batches + len(x)
    print(test_batches)

    train_gen = chain(iter_list_train)
    val_gen = chain(iter_list_val)
    test_gen = chain(iter_list_test)
    print('78')
    # MLP Model
    model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 64, activation='leaky_relu'),
    tf.keras.layers.Dense(units = 64, activation='leaky_relu'),
    tf.keras.layers.Dense(units=12),
    ])


    callbacks = [
      EarlyStopping(monitor='val_loss', patience=3,  mode='min',verbose=1),
      ReduceLROnPlateau(factor=0.15, patience=2, min_lr=1e-9, verbose=1),
      ModelCheckpoint('MLP_{}_gaussian.h5'.format(ws), verbose=1, save_best_only=True, save_weights_only=True) ]

    model.compile(loss=tf.keras.losses.Huber(),
                optimizer=tf.optimizers.SGD(learning_rate=1e-4, momentum = 0.9),
                metrics=[tf.metrics.MeanAbsoluteError()])

    history = model.fit(train_gen, steps_per_epoch = train_batches, epochs=25, validation_data = val_gen, validation_steps = val_batches, callbacks=callbacks)
    model.evaluate(test_gen, steps = test_batches)

2021-12-05 03:41:58.789237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14659 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0


KeyboardInterrupt: 

In [ ]:
# model.built = True
model.load_weights('LSTM_ws_15_sr_2.h5')
# model.evaluate(test_gen, steps = test_batches)

In [ ]:
mpl.rcParams['figure.figsize'] = (8,8)
day = 100
grouped_imputed_df = imputed_df.groupby(['day'], sort = False, as_index = False)
def plot(i):
    pd.Series(x.iloc[i].values , index=range(len(full_test_dfs[i])+10,len(full_test_dfs[i])+121,10)).plot()
    (grouped_imputed_df.get_group(test_days_list[i])['GHI'].reset_index(drop = True)).plot()

In [ ]:
def get_test_predictions(model, ws,features_list,sr) :

  X = np.zeros((300,ws,len(features_list)))
  for i in range(300):  
    X[i] = np.array(full_test_dfs[i][(len(full_test_dfs[i])-1)%sr::sr][-ws:][features_list])
  output = model.predict(X)
  output = output.reshape(300,12)
  output = pd.DataFrame(output)
#     print(output)
  output = output*train_std.loc['GHI'] + train_mean.loc['GHI']
  for i in range(300):
    for j in range(4):
      if(output.iloc[i][j] < -1):
          output.iloc[i][j] = -0.8
  create_submission(pd.DataFrame(np.array(output)),'predictions_ws_{}sr_{}cols_{}.csv'.format(ws,sr,len(features_list)))
  return output

### LSTM

In [70]:
def get_data(ws, dataframe):
    df = dataframe.copy(deep = False)
    df.reset_index(drop = True, inplace = True)
#     imgs_df = df[image_type[0]]
#     df = df[dataframe_cols_list]
#     image_array = np.array([json_load[path] for path in imgs_df ])
    data = []
#     images = []
    targets = []
    grouped_df = df.groupby(['day'], sort = False, as_index = False)
    total_window_size = ws + 12
    for day in df['day'].unique():
        day_size = grouped_df.get_group(day)['GHI'].count()
#         offset = grouped_df.get_group(day).index.values[0]
        day_df = grouped_df.get_group(day).reset_index(drop = True)
        day_df.drop('day',axis = 1,inplace = True)
        if(day_size<total_window_size):
            pass
        else:
            for i in range(day_size - total_window_size + 1):
                targets.append(np.array(day_df['GHI'].iloc[list(range(ws+i,ws+i+12))]))
#                 images.append(image_array[offset+i:offset+ws+i])
                data.append(np.array((day_df.iloc[list(range(i,ws+i))])))
    return np.array(data),np.array(targets)

In [46]:
test_df.columns

Index(['GHI', 'DNI', 'Zenith Angle', 'Azimuth Angle', 'Dry Temp', 'Wet Temp',
       'Dew Temp', 'RH', 'TCC', 'Pressure', 'Precipitation', 'Snow Depth',
       'Moisture', 'Albedo', 'Time', 'day', 'Declination Angle',
       'distance factor', 'Wind_x', 'Wind_y', 'Clear Sky GHI'],
      dtype='object')

In [71]:
# bs = 128
train_gen = get_data(12,train_df.drop(['Zenith Angle','Snow Depth','Albedo','Time'],axis = 1)[::10])
val_gen = get_data(12,val_df.drop(['Zenith Angle','Snow Depth','Albedo','Time'],axis = 1)[::10])
test_gen = get_data(12,test_df.drop(['Zenith Angle','Snow Depth','Albedo','Time'],axis = 1)[::10])

In [72]:
X_train, y_train = train_gen
X_val, y_val = val_gen
X_test, y_test = test_gen

In [73]:
bs = 128
train_gener = DATALOADER(X_train, y_train,bs)
val_gener = DATALOADER(X_val, y_val,bs)
test_gener = DATALOADER(X_test, y_test,bs)

In [76]:
i = i + 1
model = tf.keras.Sequential([
tf.keras.layers.LSTM(64,return_sequences = True,input_shape = (12,16)),
tf.keras.layers.Bidirectional(LSTM(64,return_sequences = True)),
tf.keras.layers.LSTM(64),
tf.keras.layers.Dense(units=12),
])

# go_backwards=True
callbacks = [
  EarlyStopping(monitor='val_loss', patience=200,  mode='min',verbose=1),
  ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-9, verbose=1),
  ModelCheckpoint('LSTM_final.h5', verbose=1, save_best_only=True, save_weights_only=True) ]

# lr_schedule = keras.callbacks.LearningRateScheduler(
# lambda epoch: 1e-8 * 10**(epoch / 10))

model.compile(loss=tf.keras.losses.Huber(),
            optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            metrics=[tf.metrics.MeanAbsoluteError()])

In [77]:
history = model.fit(train_gener,steps_per_epoch = len(y_train)//bs, epochs=80, validation_data = val_gener , validation_steps = len(y_val)//bs,callbacks = [callbacks])
# model.evaluate(test_gen, steps = len(test_df)/bs)

Epoch 1/80
390/390 [==============================] - 5s 7ms/step - loss: 0.1569 - mean_absolute_error: 0.3963 - val_loss: 0.1358 - val_mean_absolute_error: 0.3448

Epoch 00001: val_loss improved from inf to 0.13579, saving model to LSTM_final.h5
Epoch 2/80
390/390 [==============================] - 2s 6ms/step - loss: 0.1322 - mean_absolute_error: 0.3449 - val_loss: 0.1337 - val_mean_absolute_error: 0.3405

Epoch 00002: val_loss improved from 0.13579 to 0.13374, saving model to LSTM_final.h5
Epoch 3/80
390/390 [==============================] - 2s 6ms/step - loss: 0.1282 - mean_absolute_error: 0.3367 - val_loss: 0.1371 - val_mean_absolute_error: 0.3622

Epoch 00003: val_loss did not improve from 0.13374
Epoch 4/80
390/390 [==============================] - 2s 6ms/step - loss: 0.1259 - mean_absolute_error: 0.3337 - val_loss: 0.1357 - val_mean_absolute_error: 0.3443

Epoch 00004: val_loss did not improve from 0.13374
Epoch 5/80
390/390 [==============================] - 2s 6ms/step - lo

KeyboardInterrupt: 

In [80]:
model.load_weights('LSTM_final.h5')
model.evaluate(test_gener, steps = len(test_df)/bs)

1481/1481 [==============================] - 4s 3ms/step - loss: 0.1397 - mean_absolute_error: 0.3507


[0.13971126079559326, 0.35065022110939026]

In [ ]:
# from matplotlib import pyplot
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss vs accuracy')
# plt.xlabel('epoch')
# plt.legend(['loss', 'val_loss'], loc='upper right')
# plt.show() 

In [ ]:
get_test_predictions(model,12,l[2],10)

In [ ]:
sr = 10
for ws in [12]:
    grouped_df1 = train_df.groupby(['day'], sort = False, as_index = False)
    iter_list_train = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in train_df['day'].unique() ]

    grouped_df1 = val_df.groupby(['day'], sort = False, as_index = False)
    iter_list_val = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in val_df['day'].unique() ]

    grouped_df1 = test_df.groupby(['day'], sort = False, as_index = False)
    iter_list_test = [ WindowGenerator(input_width=ws, label_width = 12, shift=10//sr, label_shift = 10//sr,train_df = grouped_df1.get_group(day)[::sr][l[2]],label_columns=['GHI']).train for day in test_df['day'].unique() ]
    # test_generator = itertools.chain.from_iterable( iter_list )

    train_batches = 0
    for x in iter_list_train:
        train_batches = train_batches + len(x)
    print(train_batches)
    val_batches = 0
    for x in iter_list_val:
        val_batches = val_batches + len(x)
    print(val_batches)

    test_batches = 0
    for x in iter_list_test:
        test_batches = test_batches + len(x)
    print(test_batches)

    train_gen = chain(iter_list_train)
    val_gen = chain(iter_list_val)
    test_gen = chain(iter_list_test)